## byom

In [ ]:
%%sh
#copy the model file from bert-serve pretrained
mkdir -p export/Servo
cp -r ../models/export-model/* export/Servo/

In [8]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [22]:
#upload model file

In [9]:
import sagemaker

sagemaker_session = sagemaker.Session()
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

# Deploy use pretrained model

In [23]:
from sagemaker import get_execution_role

role = get_execution_role()

image = '847380964353.dkr.ecr.us-east-1.amazonaws.com/bert-tf-sentiment-analysis:latest'

In [26]:
from sagemaker.tensorflow.model import TensorFlowModel
sagemaker_model = TensorFlowModel(model_data = 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz',
                                  role = role,
                                  image = image,
                                  entry_point = 'bert/run_classifier.py'
                                 )

2.1.0 is the latest version of tensorflow that supports Python 2. Newer versions of tensorflow will only be available for Python 3.Please set the argument "py_version='py3'" to use the Python 3 tensorflow image.
No framework_version specified, defaulting to version 1.11. This is not the latest supported version. If you would like to use version 2.1.0, please add framework_version=2.1.0 to your constructor.


In [27]:
%%time
predictor = sagemaker_model.deploy(initial_instance_count=1,
                                          instance_type='ml.m4.xlarge')

-----------------!CPU times: user 428 ms, sys: 11.4 ms, total: 440 ms
Wall time: 8min 32s


# predict

In [30]:
import sys
sys.path.append("./bert")

import tokenization
from extract_features import InputExample, convert_examples_to_features
import numpy as np
import requests
import os
import time

vocab_file = os.environ.get('vocab_file', './bert/pretrain_model/vocab.txt')
max_token_len = os.environ.get('max_token_len', 128)


def preprocess(text):
    text_a = text
    example = InputExample(unique_id=None, text_a=text_a, text_b=None)
    tokenizer = tokenization.FullTokenizer(
        vocab_file=vocab_file, do_lower_case=True)
    feature = convert_examples_to_features([example], max_token_len, tokenizer)[0]
    input_ids = np.reshape([feature.input_ids], (1, max_token_len))
    return {
        "inputs": {"input_ids": input_ids.tolist()}
    }

In [31]:
from sagemaker.predictor import json_serializer, json_deserializer

text = "测试一下啦！"
data = preprocess(text)

predictor.accept = 'application/json'
predictor.content_type = 'application/json'

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

start = time.time()
resp = predictor.predict(data)
end = time.time()
pro_0, pro_1 = resp['outputs'][0]
print(f"negative pro:{pro_0} positive pro:{pro_1} time consuming:{int((end - start) * 1000)}ms")



INFO:tensorflow:*** Example ***
INFO:tensorflow:unique_id: None
INFO:tensorflow:tokens: [CLS] 测 试 一 下 啦 ！ [SEP]
INFO:tensorflow:input_ids: 101 3844 6407 671 678 1568 8013 102 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:input_type_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
negative pro:

In [21]:
sagemaker.Session().delete_endpoint(predictor.endpoint)